# Assignment — Multi-hop Reasoning on Knowledge graphs

Knowledge graph embedding allows predicting missed links in the graph. However, it does not allow to answer to complex logical queries.

For example, one can want to answer `Where did Canadian citizens with Turing Award graduate?` Such question can be decomposed into several smaller questions and construct DAG (directed acyclic graph) of logical operations.


<img src='https://github.com/netspractice/network-science/raw/main/assignment_multihop/model.png' width=700>

In [ ]:
!pip install torchkge -q

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torchkge import KnowledgeGraph
from torchkge.utils import Trainer
from torchkge.models.bilinear import RESCALModel
from torchkge.utils import MarginLoss
import requests
import pandas as pd
import numpy as np
from zlib import adler32
from tqdm.notebook import trange

### Task 1. Beam search with RESCAL (3 point)

Beam-search is a technique of generating most probable sequences.

<img src='https://github.com/netspractice/network-science/raw/main/assignment_multihop/beamsearch.png' width=600>

It works as follows:
1. Start from some root (`<START>` token on image)
2. Predict subsequent tokens
3. Select `k` most probable subsequences from generated
4. Repeat the procedure

In the current task, we will apply it to the query represented in the sequential form. We will work with the queries that can be represented in conjunctive normal form. For example, we have a query `What country was replaced by Canada neighbours`. `Canada neighbors` can be found by prediction tail `t` for query `h = "Canada"`, `r = "shares border with"`. And `What country was replaced by` could be represented as a tail for query `h = t`, `r = "replaces"`. So our query can be decomposed into two smaller ones.

In [ ]:
url = 'https://raw.githubusercontent.com/netspractice/network-science/main/datasets/countries_edges.tsv'
open('countries_edges.tsv', 'wb').write(requests.get(url).content)
url = 'https://raw.githubusercontent.com/netspractice/network-science/main/datasets/countries_entities.tsv'
open('countries_entities.tsv', 'wb').write(requests.get(url).content)
url = 'https://raw.githubusercontent.com/netspractice/network-science/main/datasets/countries_relations.tsv'
open('countries_relations.tsv', 'wb').write(requests.get(url).content);

edges = pd.read_csv('countries_edges.tsv', sep='	').values
entity_labels = pd.read_csv('countries_entities.tsv', sep='	', index_col=0).label.values
relation_labels = pd.read_csv('countries_relations.tsv', sep='	', index_col=0).label.values

edges_labeled = np.stack([entity_labels[edges[:, 0]], 
                          entity_labels[edges[:, 1]], 
                          relation_labels[edges[:, 2]]], axis=1)

df = pd.DataFrame(edges_labeled, columns=['h', 't', 'r'])[['h', 'r', 't']]
df.head()

Firstly, we need to check if the answer is in the given dataset.

In [ ]:
neighbors = set(df[(df.h == 'Canada') & (df.r == "shares border with")].t)

In [ ]:
def find_replaces(df, neighbors):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
el = find_replaces(df, neighbors).pop()
assert adler32(el.encode()) == 2730560188
print(el)

We can find an answer in our dataset, but for complex queries or incomplete graphs, such a task could be very hard. So we can work with knowledge graph embedding models to solve it. Firstly, let us initialize the knowledge graph dataset from torchkge.

In [ ]:
kg = KnowledgeGraph(pd.DataFrame(edges_labeled, columns=['from', 'to', 'rel']))

Secondly, we need to train the fine embedding model. We will use `RESCALModel` from torchkge. Similarly to the TransE model from the previous seminar, it learns two embedding tensors. However, instead of embed relations, it learns the projection matrix for each relation.

In [ ]:
model = RESCALModel(emb_dim=128, n_entities=kg.n_ent, n_relations=kg.n_rel)
criterion = MarginLoss(margin=0.5)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

trainer = Trainer(
    model, criterion, kg, n_epochs=50, 
    batch_size=512, optimizer=optimizer)

In [ ]:
trainer.run()

After the model trained, we need to find the top $k$ most similar tails to our head and relation.

You need to define the `find_most_similar` function that takes our trained model, knowledge graph, head and relation from a query in string form and the number of most similar items to return.

It works as follows:
1. Extract embeddings from the model using `get_embeddings` method
2. Extract vectors for head
3. Extract matrix for relation
4. Calculate predicted embedding for via matrix multiplication over head vector and relation matrix
5. Normalize predicted vector
6. Calculate cosine similarity between predicted embedding and each entity from entity embedding matrix (dot product)

Return np.array with indices of the top k most similar entities and np.array with corresponding similarities sorted in descending order

In [ ]:
def find_most_similar(model, kg, head, relation, k):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
ids, sims = find_most_similar(model, kg, "Canada", "shares border with", 5)
assert len(ids) == 5
assert ((sims[:-1] - sims[1:]) >= 0).mean() == 1

Now we can try to answer to our query in two steps.

In [ ]:
# 1
ids, sims = find_most_similar(model, kg, "Canada", "shares border with", 5)

# 2
ix2ent = list(kg.ent2ix.keys())
results = []
for i in ids:
    idx, s = find_most_similar(model, kg, ix2ent[i], "replaces", 5)
    score_matrix = np.outer(sims, s).flatten()
    topk = score_matrix.argsort()[-5:]
    results.extend(zip(idx[topk % 5], score_matrix[topk]))
results_topk = sorted(results, key=lambda x: x[1])[-5:]
results_topk_entities = [ix2ent[j] for j, _ in results_topk]

assert 2730560188 in [adler32(i.encode()) for i in results_topk_entities]
print('\n'.join(results_topk_entities))

### Task 2. Query2Box (7 points)

Source http://snap.stanford.edu/query2box/

Query2Box additionally allow to model union (disjunction) queries.


The general idea behind Query2Box is to model sets as boxes. If an entity is in the set, then the corresponding embedding should lie inside the query box. The box is defined by the vector of centre and offset.

The projection (existential operator) works similarly to the translation models: the model sums the centres and offsets.
The intersection of the boxes could be found by performing attention over box queries. Offsets are calculated using DeepSets over the boxes and are shrunk with the sigmoid function.

A simple geometric union of the boxes could be a bad idea because query boxes could lie in different places of our space. So, before doing union, the authors propose transforming our query to the disjunctive normal form (DNF).
It allows to perform all box logic with projection and intersection operators and, finally, found the best entities close to one of the resulting boxes.

<img src='http://snap.stanford.edu/query2box/dnf.png' width=700>




Let us download the realized models from https://github.com/snap-stanford/KGReasoning.

In [ ]:
url = 'https://raw.githubusercontent.com/snap-stanford/KGReasoning/main/models.py'
with open("models.py", "wb") as f:
    f.write(requests.get(url).content)
    
url = 'https://raw.githubusercontent.com/snap-stanford/KGReasoning/main/dataloader.py'
with open("dataloader.py", "wb") as f:
    f.write(requests.get(url).content)
    
url = 'https://raw.githubusercontent.com/snap-stanford/KGReasoning/main/util.py'
with open("util.py", "wb") as f:
    f.write(requests.get(url).content)

In [ ]:
import models

Firstly, we define types of queries we will prefer to handle.

* 1p is one-projection query
* 2p is two-projection query
* 2i is two-intersection query
* 2u-DNF is a disjunctive normal form, i.e. union (disjunction) between two 1p queries (Q2B)

Naming convention: p projection, i intersection, n negation, u union.

In [ ]:
query_name_dict = {
    ('e',('r',)): '1p', 
    ('e', ('r', 'r')): '2p',
    (('e', ('r',)), ('e', ('r',))): '2i',
    (('e', ('r',)), ('e', ('r',)), ('u',)): '2u-DNF',
}

Here is e — entity, r — relation.

Next, we need to generate the disjunctive examples for training.

We will generate union pairs only for relation type `shares border with`.
`generate_queries_disjunction` takes the data frame with triplets, entity to index converter, relation to index converter, number of elements in subsample and random_state for sampling.

1. Generate one-hop queries
2. Generate two samples from step 1 using `df.sample(n, random_state=random_state)`
3. zip one-hop queries and construct union query (e.g. `((187, (41, )), (1071, (41, )), (-1,))`)
4. construct answers as unions of answers for one-hop queries

In [ ]:
def generate_queries_disjunction(df, ent2id, rel2id, n, random_state=0):
    # YOUR CODE HERE
    raise NotImplementedError()

Here is an example how the query and answer look like
```python
query, answer = generate_queries_disjunction(df, kg.ent2ix, kg.rel2ix, 1, 0)
assert query == [(((36, (41,)), (616, (41,)), (-1,)), (('e', ('r',)), ('e', ('r',)), ('u',)))]
assert answer == {((36, (41,)), (616, (41,)), (-1,)): {1026, 611, 1157, 1574, 710, 1256, 1257, 1676, 1560, 669, 216}}
```

In [ ]:
train_queries_dis = []
train_answers_dis = {}
for i in trange(500):
    tq, ta = generate_queries_disjunction(df, kg.ent2ix, kg.rel2ix, 300, random_state=2 * i)
    train_queries_dis.extend(tq)
    train_answers_dis.update(ta)

In [ ]:
assert train_queries_dis[0][1] == (('e', ('r',)), ('e', ('r',)), ('u',))
assert (((129, (41,)), (1706, (41,)), (-1,)), (('e', ('r',)), ('e', ('r',)), ('u',))) in train_queries_dis
assert sum(train_answers_dis[(((129, (41,)), (1706, (41,)), (-1,)))]) == 2267

Initialize model, optimizer and train iterator.

In [ ]:
q2b_model = models.KGReasoning(
    nentity=kg.n_ent,
    nrelation=kg.n_rel,
    hidden_dim=800,
    gamma=24,
    geo="box",
    use_cuda=False,
    box_mode=('relu', 0.05),
    beta_mode=None,
    test_batch_size=128,
    query_name_dict=query_name_dict
)

optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, q2b_model.parameters()), 
    lr=0.01
)

train_path_iterator = SingledirectionalOneShotIterator(DataLoader(
    TrainDataset(train_queries_dis, kg.n_ent, kg.n_rel, 128, train_answers_dis),
    batch_size=512,
    shuffle=True,
    num_workers=4,
    collate_fn=TrainDataset.collate_fn
))

Now, we will try to answer the question `What countries share a border with Canada or Mexico?`.

In [ ]:
set(df[(df.h == 'Mexico') & (df.r == 'shares border with')].\
    append(df[(df.h == 'Canada') & (df.r == 'shares border with')]).t)


In [ ]:
for i in trange(50):
    q2b_model.train_step(q2b_model, optimizer, train_path_iterator, args,i)

In [ ]:
ans = predict_kg_reasoning(
    q2b_model,
    kg,
    [kg.ent2ix['Canada'], kg.rel2ix['shares border with'], kg.ent2ix['Mexico'], kg.rel2ix['shares border with'], -1],
    (('e', ('r',)), ('e', ('r',)), ('u',)), k=10, ix2ent=ix2ent)
hashed = [adler32(i.encode()) for i in ans]
assert 1847527621 in hashed
assert 1897990456 in hashed
assert 131007068 in hashed
assert 295175058 in hashed
assert 292684689 in hashed
print('\n'.join(ans))